In [ ]:
from scipy import linalg
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import colors
import pickle
from random import sample
import matplotlib.lines as mlines
%matplotlib inline

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


# #############################################################################
# Colormap
cmap = colors.LinearSegmentedColormap(
    'red_blue_classes',
    {'red': [(0, 1, 1), (1, 0.7, 0.7)],
     'green': [(0, 0.7, 0.7), (1, 0.7, 0.7)],
     'blue': [(0, 0.7, 0.7), (1, 1, 1)]})
plt.cm.register_cmap(cmap=cmap)



# #############################################################################
# Generate datasets
def dataset_fixed_cov(n,dim):
    '''Generate 2 Gaussians samples with the same covariance matrix'''
    np.random.seed(0)
    C = np.array([[0., -0.23], [0.83, 0.23]]) 
    X = np.r_[np.dot(3*np.random.randn(n, dim), C) + np.array([-1.0, -1.0]),
              np.dot(3*np.random.randn(n, dim), C) + np.array([1.0, 1.0])]
    y = np.hstack((np.zeros(n), np.ones(n)))
    return X, y


def dataset_cov(n,dim):
    '''Generate 2 Gaussians samples with different covariance matrices'''
    np.random.seed(0)

    C = np.array([[0.0, -1.0], [2.5, 1.7]]) * 1.   #[[0., -1.], [2.5, 1.7]]) * 2.    + np.array([0,0])] : 0.9
    CT = np.array([[0.0, 2.5], [-1.0, 1.7]])

    X = np.r_[np.dot(np.random.randn(n, dim), C),
              np.dot(np.random.randn(n, dim), CT) + np.array([0, 0])]
    y = np.hstack((np.zeros(n), np.ones(n)))
    return X, y


def plot_raw_data(X, y, X_test, y_test):
    # scatter plot the entire training and testing cases
    splot = plt.subplot(1, 2, 1)

    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(10, 4)

    # splot = plt.subplot(2, 2,figsize=(15,15))
    # splot = plt.subplots(figsize=(3, 4))


    plt.title('Training, %i samples' %X.shape[0])
    plt.ylabel('Data with\n different covariance')
    X_neg = X[y == 0]
    X_pos = X[y == 1]

    plt.scatter(X_neg[:, 0], X_neg[:, 1], marker='.', color='red')
    plt.scatter(X_pos[:, 0], X_pos[:, 1], marker='.', color='blue')

    nx, ny = 900, 300
    plt.xlim(-10, 10)
    plt.ylim(-10, 10)
    x_min, x_max = plt.xlim()
    y_min, y_max = plt.ylim()
        
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, nx),
                     np.linspace(y_min, y_max, ny))

    TPs = mlines.Line2D([], [], color='blue', marker='.', linestyle='None',
                      markersize=5, label='Positives')
    TNs = mlines.Line2D([], [], color='red', marker='.', linestyle='None',
                      markersize=5, label='Negatives')
        
    plt.legend(handles=[TPs, TNs],loc='lower right')

    splot = plt.subplot(1, 2, 2)
        
    plt.title('Testing, %i samples' %X_test.shape[0])
    plt.ylabel('Data with\n different covariance')
    X_neg = X_test[y_test == 0]
    X_pos = X_test[y_test == 1]

    plt.scatter(X_neg[:, 0], X_neg[:, 1], marker='.', color='red')
    plt.scatter(X_pos[:, 0], X_pos[:, 1], marker='.', color='blue')

    nx, ny = 900, 300
    plt.xlim(-10, 10)
    plt.ylim(-10, 10)
    x_min, x_max = plt.xlim()
    y_min, y_max = plt.ylim()
        
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, nx),
                     np.linspace(y_min, y_max, ny))

    TPs = mlines.Line2D([], [], color='blue', marker='.', linestyle='None',
                          markersize=5, label='Positives')
    TNs = mlines.Line2D([], [], color='red', marker='.', linestyle='None',
                          markersize=5, label='Negatives')
        
    plt.legend(handles=[TPs, TNs],loc='lower right')
   
    return plt


def plot_hyperplane_data(lda, X, y, X_test, y_pred, y_truth):
    # scatter plot train and test images with class separing hyperplane
    splot = plt.subplot(1, 2, 1)
    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(10, 4)

    plt.title('Training, %i samples' %X.shape[0])
    plt.ylabel('Data with\n fixed covariance')
    X_neg = X[y == 0]
    X_pos = X[y == 1]

    plt.scatter(X_neg[:, 0], X_neg[:, 1], marker='.', color='red')
    plt.scatter(X_pos[:, 0], X_pos[:, 1], marker='.', color='blue')

    nx, ny = 600, 300
    plt.xlim(-10, 10)
    plt.ylim(-10, 10)
    x_min, x_max = plt.xlim()
    y_min, y_max = plt.ylim()
        
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, nx),
                         np.linspace(y_min, y_max, ny))
    Z = lda.predict_proba(np.c_[xx.ravel(), yy.ravel()])
    Z = Z[:, 1].reshape(xx.shape)
    plt.pcolormesh(xx, yy, Z, cmap='red_blue_classes',
                   norm=colors.Normalize(0., 1.), zorder=0, shading='auto')
    plt.contour(xx, yy, Z, [0.5], linewidths=2., colors='white')


    xx_train = xx
    yy_train = yy
    Z_train = Z

    splot = plt.subplot(1, 2, 2)
        
    plt.ylabel('Data with\n fixed covariance')

    tp = (y_truth == y_pred)  # True Positive
    tp0, tp1 = tp[y_truth == 0], tp[y_truth == 1]
    X0, X1 = X_test[y_truth == 0], X_test[y_truth == 1]
    X0_tp, X0_fp = X0[tp0], X0[~tp0]
    X1_tp, X1_fp = X1[tp1], X1[~tp1]

    plt.scatter(X0_tp[:, 0], X0_tp[:, 1], marker='.', color='red')
    plt.scatter(X0_fp[:, 0], X0_fp[:, 1], marker='x',
            s=20, color='black')  # dark red

    # class 1: dots
    plt.scatter(X1_tp[:, 0], X1_tp[:, 1], marker='.', color='blue')
    plt.scatter(X1_fp[:, 0], X1_fp[:, 1], marker='x',
            s=20, color='black')  # dark blue

    # class 0 and 1 : areas
    nx, ny = 200, 100
    plt.xlim(-10, 10)
    plt.ylim(-10, 10)
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, nx),
                     np.linspace(y_min, y_max, ny))
    Z = lda.predict_proba(np.c_[xx.ravel(), yy.ravel()])
    Z = Z[:, 1].reshape(xx.shape)
    plt.pcolormesh(xx_train, yy_train, Z_train, cmap='red_blue_classes',
               norm=colors.Normalize(0., 1.), zorder=0, shading='auto')
    plt.contour(xx_train, yy_train, Z_train, [0.5], linewidths=2., colors='white')

    plt.title('Testing, ' "{0:.2%} Accuracy".format(sum(tp)/len(tp)))

    TPs = mlines.Line2D([], [], color='blue', marker='.', linestyle='None',
                      markersize=5, label='True Positive')
    TNs = mlines.Line2D([], [], color='red', marker='.', linestyle='None',
                          markersize=5, label='True Negatives')
    misclassified = mlines.Line2D([], [], color='black', marker='x', linestyle='None',
                      markersize=5, label='missclassified')
    plt.legend(handles=[TPs, TNs, misclassified],loc='lower right')

    
    return splot, sum(tp)/len(tp)

In [ ]:
####################################
### Linear Discriminant Analysis ###
####################################


train_size = 200        # number of positive and negative training samples should be > 3
test_size = 50          # number of positive and negative testing samples should be > 3
varData_train = dataset_fixed_cov(train_size,2)
varData_test = dataset_fixed_cov(test_size,2)
splot = plot_raw_data(varData_train[0], varData_train[1], varData_test[0], varData_test[1])
splot.tight_layout()
splot.subplots_adjust(top=0.9)
splot.show()

In [ ]:
train_ratio = 0.1     # ratio of  data to use for training the classifier. should be in (0,1]
all_indices = list(range(varData_train[0].shape[0]))
all_indices_positive = [i for i, x in enumerate(varData_train[1]) if x]
all_indices_negative = [i for i, x in enumerate(varData_train[1]) if not x]
train_size = int(len(all_indices_positive)*train_ratio)
if(train_size < 2):
    train_size = 2


train_index_positive = sample(all_indices_positive,train_size)
train_index_negative = sample(all_indices_negative,train_size)
train_index = train_index_negative + train_index_positive

X_train = varData_train[0][train_index]
y_train = varData_train[1][train_index]
X_test = varData_test[0]
y_test = varData_test[1]

lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=True)
y_pred = lda.fit(X_train, y_train).predict(X_test)
splot,accuracy = plot_hyperplane_data(lda, X_train, y_train, X_test, y_pred, y_test)

plt.tight_layout()
plt.subplots_adjust(top=0.9)

In [ ]:
#######################################
### Quadratic Discriminant Analysis ###
#######################################


train_size = 150        # number of positive and negative training samples should be > 3
test_size = 50          # number of positive and negative testing samples should be > 3

varData_train = dataset_cov(train_size,2)
varData_test = dataset_cov(test_size,2)
splot = plot_raw_data(varData_train[0], varData_train[1], varData_test[0], varData_test[1])
splot.tight_layout()
splot.subplots_adjust(top=0.9)
splot.show()

In [ ]:
train_ratio = 0.1     # ratio of  data to use for training the classifier. should be in (0,1]
all_indices = list(range(varData_train[0].shape[0]))
all_indices_positive = [i for i, x in enumerate(varData_train[1]) if x]
all_indices_negative = [i for i, x in enumerate(varData_train[1]) if not x]
train_size = int(len(all_indices_positive)*train_ratio)
if(train_size < 3):
    train_size = 3


train_index_positive = sample(all_indices_positive,train_size)
train_index_negative = sample(all_indices_negative,train_size)
train_index = train_index_negative + train_index_positive


X_train = varData_train[0][train_index]
y_train = varData_train[1][train_index]
X_test = varData_test[0]
y_test = varData_test[1]

qda = QuadraticDiscriminantAnalysis(store_covariance=True)
y_pred = qda.fit(X_train, y_train).predict(X_test)
splot,accuracy = plot_hyperplane_data(qda, X_train, y_train, X_test, y_pred, y_test)

plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()